# 合并data_to_fc/feature训练数据集（e39_0-6999）,测试数据集（e39_7000-7999)

In [2]:
import numpy as np
import os
import socket

In [3]:
bits = 64
batch_size = 32
whole_train_sam_num = 7000 # 完整的训练集样本数
smaller_train_sam_num = 3000 # 减小内存消耗的训练集样本数
smallerer_train_sam_num = 1000

host_name = socket.gethostname()
if host_name == 'DESKTOP-PLRL7TK':
    dir_intermediate = ''
elif host_name == 'DESKTOP-6FOH47P':
    dir_intermediate = 'F:\\Projects\\python\\PQ\\intermediate8dbfc1\\'
elif host_name == 'jm-System-Product-Name':
    dir_intermediate = '/data/hdr/transformer_data/intermediate/'+str(bits)+'/'
    dir_train = os.path.join('/data/hdr/transformer_data/joined', 'train', 'f'+str(bits))
    dir_test = os.path.join('/data/hdr/transformer_data/joined', 'test', 'f'+str(bits))
    # dir1 = '/data/hdr/transformer_data/joined/'
    # dir2 = '/data/hdr/transformer_data/model/'
else:
    raise NameError("You are running the script in a new computer, please define dir_intermediate")



In [4]:
def join_from_intermediate(dir_intermediate, dir_train, bits, intermediate_name, sam_num, trainortest):#不需要合并第一维
    #sam_num:合并的样本数;trainortest:合成训练集填"train",测试集填"test"
    linearinpath0= os.path.join(dir_intermediate, str(bits), intermediate_name+'_f%i_e39_0.npy' % bits)#例：此处intermediate_name为linearin
    linearin0 = np.load(linearinpath0)
    print("合并第一维前大小：", linearin0.shape)
    #把第一维合并
    # linearin0_join1 = np.reshape(linearin0, (-1, linearin0.shape[-1]))
    # print("合并第一维后大小：", linearin0_join1.shape)
    if trainortest == "test":
        add = 7000
    else:
        add = 0
    for i in range(1+add, sam_num+add):
        linearinpath1= os.path.join(dir_intermediate, str(bits), intermediate_name+'_f%i_e39_%i.npy' % (bits, i) )
        linearin1 = np.load(linearinpath1)
        linearin1 = np.reshape(linearin1, (-1, linearin1.shape[-1]))
        if linearin1.shape[0]!=1024:
            print("i",str(i),",shape",str(linearin1.shape[0]))
        linearin0 = np.append(linearin0, linearin1, axis=0)
    print(linearin0.shape)
    np.save(os.path.join(dir_train, intermediate_name+trainortest+'_f%i_sam%i.npy' % (bits,whole_train_sam_num)), linearin0) 
        

In [4]:
def join_from_joined(dir_train, linear_name, bits, joined_sam_num, smaller_sam_sum, batch_size):# 从已合成训练/测试集中提取更小的训练测试集
    # 例:linear_name:'ex_linear1in'
    # joined_sam_num：已合成训练/测试集的样本数
    # sam_num：提取出的训练/测试集的样本数
    ex_linear1in_whole = np.load(os.path.join(dir_train, linear_name + '_train_f%i_sam%i.npy' % (bits,joined_sam_num)))
    print(ex_linear1in_whole.shape)
    ex_linear1in_smaller = ex_linear1in_whole[np.ix_(range(smaller_sam_sum*batch_size), range(ex_linear1in_whole.shape[1]))]
    print(ex_linear1in_smaller.shape)
    np.save(os.path.join(dir_train, '%s_train_f%i_sam%i.npy' % (linear_name, bits, smaller_sam_sum)), ex_linear1in_smaller) 

In [3]:
data_to_fcpath0= os.path.join(dir_intermediate, 'data_to_fc_f%i_e39_0.npy' % bits)
data_to_fc0 = np.load(data_to_fcpath0, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
# 合并data_to_fc训练数据集（e39_0-6999）
for i in range(1,whole_train_sam_num):
    data_to_fcpath1= os.path.join(dir_intermediate, 'data_to_fc_f%i_e39_%i.npy' % (bits, i) )
    data_to_fc1 = np.load(data_to_fcpath1, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
    data_to_fc0 = np.append(data_to_fc0, data_to_fc1, axis=0)
print(data_to_fc0.shape)
np.save(os.path.join(dir_train, 'data_to_fc_train_f%i_sam%i.npy' % (bits, whole_train_sam_num)), data_to_fc0) 

(224000, 2048)


In [6]:
join_from_joined(dir_train, "data_to_fc", bits, whole_train_sam_num, smaller_train_sam_num, batch_size)

(224000, 2048)
(96000, 2048)


In [4]:
featurepath0= os.path.join(dir_intermediate, 'feature_f%i_e39_0.npy' % bits)
feature0 = np.load(featurepath0, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
# 合并feature训练数据集（e39_0-6999）
for i in range(1,whole_train_sam_num):
    featurepath1= os.path.join(dir_intermediate, 'feature_f%i_e39_%i.npy' % (bits, i) )
    feature1 = np.load(featurepath1, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
    feature0 = np.append(feature0, feature1, axis=0)
print(feature0.shape)
np.save(os.path.join(dir_train, 'feature_train_f%i_sam%i.npy' % (bits, whole_train_sam_num)), feature0) 

(224000, 64)


In [7]:
join_from_joined(dir_train, "feature", bits, whole_train_sam_num, smaller_train_sam_num, batch_size)

(224000, 64)
(96000, 64)


In [6]:
featurepath0= os.path.join(dir_intermediate, 'feature_f%i_e39_7000.npy' % bits)
feature0 = np.load(featurepath0, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
# 合并feature测试数据集（e39_7000-7999）
for i in range(7001,8000):
    featurepath1= os.path.join(dir_intermediate, 'feature_f%i_e39_%i.npy' % (bits, i) )
    feature1 = np.load(featurepath1, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
    feature0 = np.append(feature0, feature1, axis=0)
print(feature0.shape)
np.save(os.path.join(dir_test, 'feature_test_f%i.npy' % bits), feature0) 

(32000, 64)


In [7]:
data_to_fcpath0= os.path.join(dir_intermediate, 'data_to_fc_f%i_e39_7000.npy' % bits)
data_to_fc0 = np.load(data_to_fcpath0, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
# 合并data_to_fc测试数据集（e39_7000-7999）
for i in range(7001,8000):
    data_to_fcpath1= os.path.join(dir_intermediate, 'data_to_fc_f%i_e39_%i.npy' % (bits, i) )
    data_to_fc1 = np.load(data_to_fcpath1, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
    data_to_fc0 = np.append(data_to_fc0, data_to_fc1, axis=0)
print(data_to_fc0.shape)
np.save(os.path.join(dir_test, 'data_to_fc_test_f%i.npy' % bits), data_to_fc0) 

(32000, 2048)


In [8]:
inputpath0= os.path.join(dir_intermediate , 'input_f%i_e39_7000.npy' % bits)
input0 = np.load(inputpath0, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
print(input0.shape)

(32, 2, 16, 32)


In [9]:
# 合并input测试数据集（e39_7000-7999）
for i in range(7001,8000):
    inputpath1= os.path.join(dir_intermediate, 'input_f%i_e39_%i.npy' % (bits, i)) 
    input1 = np.load(inputpath1, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
    input0 = np.append(input0, input1, axis=0)
print(input0.shape)
np.save(os.path.join(dir_test, 'input_test_f%i.npy' % bits), data_to_fc0) 

(32000, 2, 16, 32)


# 计算xw乘法结果作为训练集

In [12]:
data_to_fc_trainpath= os.path.join(dir_train, 'data_to_fc_train_f%i_sam%i.npy' % (bits, whole_train_sam_num))
data_to_fc_train = np.load(data_to_fc_trainpath, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
feature_trainpath= os.path.join(dir_train, 'feature_train_f%i_sam%i.npy' % (bits, whole_train_sam_num))
feature_train = np.load(feature_trainpath, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
biaspath= os.path.join(dir_train, 'encoder_fcb_f%i.npy' % bits)
bias = np.load(biaspath, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
weightpath= os.path.join(dir_train, 'encoder_fcw_f%i.npy' % bits)
weight = np.load(weightpath, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
data_to_fc_testpath= os.path.join(dir_test, 'data_to_fc_test_f%i.npy' % bits)
data_to_fc_test = np.load(data_to_fc_testpath, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
feature_testpath= os.path.join(dir_test, 'feature_test_f%i.npy' % bits)

In [13]:
y_trainpath = os.path.join(dir_train, 'y_train_f%i_sam%i.npy' % (bits, whole_train_sam_num))
y_testpath = os.path.join(dir_test, 'y_test_f%i_sam%i.npy' % (bits, whole_train_sam_num))

In [14]:
y_train = np.matmul(data_to_fc_train, weight)
# 计算直接乘积加bias
y_train_lastns = y_train + bias.T

y_test = np.matmul(data_to_fc_test, weight)
# 计算直接乘积加bias
y_test_lastns = y_test + bias.T

In [15]:
# 保存ytrain、ytest
np.save(y_trainpath, y_train)
np.save(y_testpath, y_test)

## 计算feature-bias作为训练集

In [9]:
data_to_fc_trainpath= os.path.join(dir_train, 'data_to_fc_train_f%i_sam%i.npy' % (bits, whole_train_sam_num))
data_to_fc_train = np.load(data_to_fc_trainpath, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
feature_trainpath= os.path.join(dir_train, 'feature_train_f%i_sam%i.npy' % (bits, whole_train_sam_num))
feature_train = np.load(feature_trainpath, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
smaller_data_to_fc_trainpath= os.path.join(dir_train, 'data_to_fc_train_f%i_sam%i.npy' % (bits, smaller_train_sam_num))
smaller_data_to_fc_train = np.load(smaller_data_to_fc_trainpath, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
smaller_feature_trainpath= os.path.join(dir_train, 'feature_train_f%i_sam%i.npy' % (bits, smaller_train_sam_num))
smaller_feature_train = np.load(smaller_feature_trainpath, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
biaspath= os.path.join(dir_train, 'encoder_fcb_f%i.npy' % bits)
bias = np.load(biaspath, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
weightpath= os.path.join(dir_train, 'encoder_fcw_f%i.npy' % bits)
weight = np.load(weightpath, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
data_to_fc_testpath= os.path.join(dir_test, 'data_to_fc_test_f%i.npy' % bits)
data_to_fc_test = np.load(data_to_fc_testpath, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
feature_testpath= os.path.join(dir_test, 'feature_test_f%i.npy' % bits)
feature_test = np.load(feature_testpath, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')

In [10]:
y_trainpath = os.path.join(dir_train, 'y_train_f%i_sam%i.npy' % (bits, whole_train_sam_num))
y_testpath = os.path.join(dir_test, 'y_test_f%i.npy' % (bits))
smaller_y_trainpath = os.path.join(dir_train, 'y_train_f%i_sam%i.npy' % (bits, smaller_train_sam_num))


In [11]:
y_train = feature_train - bias
y_test = feature_test - bias
smaller_y_train = smaller_feature_train - bias

In [13]:
# 保存ytrain、ytest
np.save(y_trainpath, y_train)
np.save(y_testpath, y_test)
np.save(smaller_y_trainpath, y_train)
